# **Exploração de dados com Apache Spark**

## **Importação da configuração inicial e bibliotecas**

In [ ]:
from ipynb.fs.full.conf_spark import *
from ipynb.fs.full.explorer_utils import *
import pandas as pd
import pyspark.sql.functions 
from pyspark.sql.functions import lit
from pyspark.sql.functions import col

## **Configuração spark**

In [ ]:
spark_settings = SettingsSpark(instance="3")
#spark_settings.log_level("DEBUG")
spark_settings.init_minio()
spark = spark_settings.builder_conf()

## **Carregamento dos arquivos para exploração e limpeza**

In [5]:
NCM = spark.read.csv("s3a://bronze/NCM.csv", sep=';', inferSchema=True, header=True, encoding="ISO-8859-1")
NCM_SH = spark.read.csv("s3a://bronze/NCM_SH.csv", sep=';', inferSchema=True, header=True, encoding="ISO-8859-1")
NBM_NCM = spark.read.csv("s3a://bronze/NBM_NCM.csv", sep=';', inferSchema=True, header=True, encoding="ISO-8859-1")
NCM_UNIDADE = spark.read.csv("s3a://bronze/NCM_UNIDADE.csv", sep=';', inferSchema=True, header=True, encoding="ISO-8859-1")
NCM_CGCE = spark.read.csv("s3a://bronze/NCM_CGCE.csv", sep=';', inferSchema=True, header=True, encoding="ISO-8859-1")
NCM_CUCI = spark.read.csv("s3a://bronze/NCM_CUCI.csv", sep=';', inferSchema=True, header=True, encoding="ISO-8859-1")
NCM_FAT_AGREG = spark.read.csv("s3a://bronze/NCM_FAT_AGREG.csv", sep=';', inferSchema=True, header=True, encoding="ISO-8859-1")
NCM_ISIC = spark.read.csv("s3a://bronze/NCM_ISIC.csv", sep=';', inferSchema=True, header=True, encoding="ISO-8859-1")
NCM_PPE = spark.read.csv("s3a://bronze/NCM_PPE.csv", sep=';', inferSchema=True, header=True, encoding="ISO-8859-1")
NCM_PPI = spark.read.csv("s3a://bronze/NCM_PPI.csv", sep=';', inferSchema=True, header=True, encoding="ISO-8859-1")
ISIC_CUCI = spark.read.csv("s3a://bronze/ISIC_CUCI.csv", sep=';', inferSchema=True, header=True, encoding="ISO-8859-1")
PAIS_BLOCO = spark.read.csv("s3a://bronze/PAIS_BLOCO.csv", sep=';', inferSchema=True, header=True, encoding="ISO-8859-1")
PAIS = spark.read.csv("s3a://bronze/PAIS.csv", sep=';', inferSchema=True, header=True, encoding="ISO-8859-1")
UF_MUN = spark.read.csv("s3a://bronze/UF_MUN.csv", sep=';', inferSchema=True, header=True, encoding="ISO-8859-1")
UF = spark.read.csv("s3a://bronze/UF.csv", sep=';', inferSchema=True, header=True, encoding="ISO-8859-1")
URF = spark.read.csv("s3a://bronze/URF.csv", sep=';', inferSchema=True, header=True, encoding="ISO-8859-1")
VIA = spark.read.csv("s3a://bronze/VIA.csv", sep=';', inferSchema=True, header=True, encoding="ISO-8859-1")


## **Visualização dos dados brutos**

In [ ]:
"""

#NCM.printSchema()
#NCM.describe().show()
print('NCM Dataset')
display(NCM.toPandas())

#NCM_SH.printSchema()
#NCM_SH.describe().show()
print('NCM_SH Dataset')
display(NCM_SH.toPandas())


#NCM_CGCE.printSchema()
#NCM_CGCE.describe().show()
print('NCM_CGCE Dataset')
display(NCM_CGCE.toPandas())#show(truncate=False)

#NCM_CUCI.printSchema()
#NCM_CUCI.describe().show()
print('NCM_CUCI Dataset')
display(NCM_CUCI.toPandas())#show(truncate=False)

#NCM_ISIC.printSchema()
#NCM_ISIC.describe().show()
print('NCM_ISIC Dataset')
display(NCM_ISIC.toPandas())#show(truncate=False)

#NCM_FAT_AGREG.printSchema()
#NCM_FAT_AGREG.describe().show()
print('NCM_FAT_AGREG Dataset')
NCM_FAT_AGREG.show(truncate=False)

#NBM_NCM.printSchema()
#NBM_NCM.describe().show()
print('NCM_NBM Dataset')
NBM_NCM.show(truncate=False)

#NCM_UNIDADE.printSchema()
#NCM_UNIDADE.describe().show()
print('NCM_UNIDADE Dataset')
NCM_UNIDADE.show(truncate=False)

#NCM_PPE.printSchema()
#NCM_PPE.describe().show()
print('NCM_PPE Dataset')
NCM_PPE.show(truncate=False)

#NCM_PPI.printSchema()
#NCM_PPI.describe().show()
print('NCM_PPI Dataset')
NCM_PPI.show(truncate=False)

#ISIC_CUCI.printSchema()
#ISIC_CUCI.describe().show()
print('ISIC_CUCI Dataset')
ISIC_CUCI.show(truncate=False)

print('PAIS_BLOCO Dataset')
PAIS_BLOCO.printSchema()
PAIS_BLOCO.describe().show()
PAIS_BLOCO.show(truncate=False)

print('PAIS Dataset')
PAIS.printSchema()
PAIS.describe().show()
PAIS.show(truncate=False)

UF_MUN.printSchema()
UF_MUN.describe().show()
UF_MUN.show(truncate=False)

URF.printSchema()
URF.describe().show()
URF.show(truncate=False)

VIA.printSchema()
VIA.describe().show()
VIA.show(truncate=False)

#spark_settings.stop()
"""

## **Normalização**

### **Normalização, exclusão de atributos de internacionalização**

In [6]:
# CO_ANO|CO_MES|  CO_NCM|CO_UNID|CO_PAIS|SG_UF_NCM|CO_VIA| CO_URF|QT_ESTAT|KG_LIQUIDO|VL_FOB|TYPE|

print("NCM",NCM.columns)
print("NCM_CUCI",NCM_CUCI.columns)
print("NCM_SH",NCM_SH.columns)
print("NCM_CGCE",NCM_CGCE.columns)
print("NCM_ISIC",NCM_ISIC.columns)
print("NCM_FAT_AGREG",NCM_FAT_AGREG.columns)
print("NBM_NCM",NBM_NCM.columns)
print("NCM_UNIDADE",NCM_UNIDADE.columns)
print("NCM_PPE",NCM_PPE.columns)
print("NCM_PPI",NCM_PPI.columns)
print("ISIC_CUCI",ISIC_CUCI.columns)
print("UF_MUN",UF_MUN.columns)
print("URF",URF.columns)
print("VIA",VIA.columns)
print("PAIS",PAIS.columns)
print("PAIS_BLOCO",PAIS_BLOCO.columns)

NCM ['CO_NCM', 'CO_UNID', 'CO_SH6', 'CO_PPE', 'CO_PPI', 'CO_FAT_AGREG', 'CO_CUCI_ITEM', 'CO_CGCE_N3', 'CO_SIIT', 'CO_ISIC_CLASSE', 'CO_EXP_SUBSET', 'NO_NCM_POR', 'NO_NCM_ESP', 'NO_NCM_ING']
NCM_CUCI ['CO_CUCI_ITEM', 'NO_CUCI_ITEM', 'CO_CUCI_SUB', 'NO_CUCI_SUB', 'CO_CUCI_GRUPO', 'NO_CUCI_GRUPO', 'CO_CUCI_DIVISAO', 'NO_CUCI_DIVISAO', 'CO_CUCI_SEC', 'NO_CUCI_SEC']
NCM_SH ['CO_SH6', 'NO_SH6_POR', 'NO_SH6_ESP', 'NO_SH6_ING', 'CO_SH4', 'NO_SH4_POR', 'NO_SH4_ESP', 'NO_SH4_ING', 'CO_SH2', 'NO_SH2_POR', 'NO_SH2_ESP', 'NO_SH2_ING', 'CO_NCM_SECROM', 'NO_SEC_POR', 'NO_SEC_ESP', 'NO_SEC_ING']
NCM_CGCE ['CO_CGCE_N3', 'NO_CGCE_N3', 'NO_CGCE_N3_ING', 'NO_CGCE_N3_ESP', 'CO_CGCE_N2', 'NO_CGCE_N2', 'NO_CGCE_N2_ING', 'NO_CGCE_N2_ESP', 'CO_CGCE_N1', 'NO_CGCE_N1', 'NO_CGCE_N1_ING', 'NO_CGCE_N1_ESP']
NCM_ISIC ['CO_ISIC_CLASSE', 'NO_ISIC_CLASSE', 'NO_ISIC_CLASSE_ING', 'NO_ISIC_CLASSE_ESP', 'CO_ISIC_GRUPO', 'NO_ISIC_GRUPO', 'NO_ISIC_GRUPO_ING', 'NO_ISIC_GRUPO_ESP', 'CO_ISIC_DIVISAO', 'NO_ISIC_DIVISAO', 'NO_ISI

In [ ]:
NCM = spark.createDataFrame(ExplorerUtils().drop_columns_matches(dataframe=NCM, patterns=["_ESP","_ING"]))
NCM.show()

NCM_CUCI= spark.createDataFrame(ExplorerUtils().drop_columns_matches(dataframe=NCM_CUCI, patterns=["_ESP","_ING"]))
NCM_CUCI.show()

NCM_SH= spark.createDataFrame(ExplorerUtils().drop_columns_matches(dataframe=NCM_SH, patterns=["_ESP","_ING"]))
NCM_SH.show()

NCM_CGCE= spark.createDataFrame(ExplorerUtils().drop_columns_matches(dataframe=NCM_CGCE, patterns=["_ESP","_ING"]))
NCM_CGCE.show()

NCM_ISIC= spark.createDataFrame(ExplorerUtils().drop_columns_matches(dataframe=NCM_ISIC, patterns=["_ESP","_ING"]))
NCM_ISIC.show()

NCM_FAT_AGREG= spark.createDataFrame(ExplorerUtils().drop_columns_matches(dataframe=NCM_FAT_AGREG, patterns=["_ESP","_ING"]))
NCM_FAT_AGREG.show()

NCM_UNIDADE= spark.createDataFrame(ExplorerUtils().drop_columns_matches(dataframe=NCM_UNIDADE, patterns=["_ESP","_ING"]))
NCM_UNIDADE.show()

NCM_PPE= spark.createDataFrame(ExplorerUtils().drop_columns_matches(dataframe=NCM_PPE, patterns=["_ESP","_ING"]))
NCM_PPE.show()

NCM_PPI= spark.createDataFrame(ExplorerUtils().drop_columns_matches(dataframe=NCM_PPI, patterns=["_ESP","_ING"]))
NCM_PPI.show()

ISIC_CUCI= spark.createDataFrame(ExplorerUtils().drop_columns_matches(dataframe=ISIC_CUCI, patterns=["_ESP","_ING"]))
ISIC_CUCI.show()

UF_MUN= spark.createDataFrame(ExplorerUtils().drop_columns_matches(dataframe=UF_MUN, patterns=["_ESP","_ING"]))
UF_MUN.show()

URF= spark.createDataFrame(ExplorerUtils().drop_columns_matches(dataframe=URF, patterns=["_ESP","_ING"]))
URF.show()

VIA= spark.createDataFrame(ExplorerUtils().drop_columns_matches(dataframe=VIA, patterns=["_ESP","_ING"]))
VIA.show()

PAIS= spark.createDataFrame(ExplorerUtils().drop_columns_matches(dataframe=PAIS, patterns=["_ESP","_ING"]))
PAIS.show()

PAIS_BLOCO= spark.createDataFrame(ExplorerUtils().drop_columns_matches(dataframe=PAIS_BLOCO, patterns=["_ESP","_ING"]))
PAIS_BLOCO.show()


#NCM = NCM.drop(NCM.filter(like='ING').columns, axis=1)
#NCM = NCM.drop(NCM.filter(like='ESP').columns, axis=1)

#NCM_SH.createOrReplaceTempView("NCM_SH")

#NCM_SH_PTBR = spark.sql("select CO_SH6, NO_SH6_POR as NO_SH6, CO_SH4, NO_SH4_POR as NO_SH4, CO_SH2, NO_SH2_POR as NO_SH2, CO_NCM_SECROM, NO_SEC_POR as NO_SEC FROM NCM_SH ")
#NCM_SH_PTBR.show(20, False)

#NCM_SH_ESP = NCM_SH[['CO_SH6', 'NO_SH6_ESP', 'CO_SH4', 'NO_SH4_ESP', 'CO_SH2', 'NO_SH2_ESP', 'CO_NCM_SECROM', 'NO_SEC_ESP']]
#NCM_SH_ESP.show()

#NCM_SH_ING = NCM_SH[['CO_SH6', 'NO_SH6_ING', 'CO_SH4', 'NO_SH4_ING', 'CO_SH2', 'NO_SH2_ING', 'CO_NCM_SECROM', 'NO_SEC_ING']]
#NCM_SH_ING.show()


### **Junção dos arquivos NCM**

In [ ]:
NCM.createOrReplaceTempView("NCM")
NCM_SH.createOrReplaceTempView("SH")
NCM_CUCI.createOrReplaceTempView("CUCI")
NCM_CGCE.createOrReplaceTempView("CGCE")
NCM_ISIC.createOrReplaceTempView("ISIC")
NCM_FAT_AGREG.createOrReplaceTempView("FAT_AGREG")
NBM_NCM.createOrReplaceTempView("NBM")
NCM_UNIDADE.createOrReplaceTempView("UNIDADE")
NCM_PPE.createOrReplaceTempView("PPE")
NCM_PPI.createOrReplaceTempView("PPI")
ISIC_CUCI.createOrReplaceTempView("ISIC_CUCI")
UF_MUN.createOrReplaceTempView("UF_MUN")
URF.createOrReplaceTempView("URF")
PAIS.createOrReplaceTempView("PAIS")
VIA.createOrReplaceTempView("VIA")
PAIS_BLOCO.createOrReplaceTempView("PAIS_BLOCO")

NCM_FULL_JOIN = spark.sql(
        """ 
            SELECT N.CO_NCM,
                   N.NO_NCM_POR as NO_NCM,
                   S.CO_SH6,
                   S.NO_SH2_POR,
                   S.NO_SH4_POR,
                   S.NO_SH6_POR AS NO_SH6,
                   lower(F.NO_FAT_AGREG) as NO_FAT_AGREG,
                   C.NO_CUCI_ITEM,
                   P.CO_PPE,
                   P.NO_PPE_MIN,
                   PI.CO_PPI,
                   PI.NO_PPI_MIN,
                   I.NO_ISIC_CLASSE
                   
              FROM NCM N
             INNER JOIN SH S
                ON N.CO_SH6 = S.CO_SH6
             INNER JOIN FAT_AGREG F
                ON N.CO_FAT_AGREG = F.CO_FAT_AGREG
             INNER JOIN CUCI C
                ON N.CO_CUCI_ITEM = C.CO_CUCI_ITEM
             INNER JOIN PPE P 
                ON N.CO_PPE = P.CO_PPE
             INNER JOIN PPI PI
                ON N.CO_PPI = PI.CO_PPI
             INNER JOIN ISIC I 
                ON N.CO_ISIC_CLASSE = I.CO_ISIC_CLASSE
         """)
               
NCM_FULL_JOIN.toPandas()

,CO_NCM,NO_NCM,CO_SH6,NO_SH2_POR,NO_SH4_POR,NO_SH6,NO_FAT_AGREG,NO_CUCI_ITEM,CO_PPE,NO_PPE_MIN,CO_PPI,NO_PPI_MIN,NO_ISIC_CLASSE
0,53012120,"Linho espadelado, mas não fiado",530121,Outras fibras têxteis vegetais; fios de papel ...,"Linho em bruto ou trabalhado, mas não fiado; e...","Linho quebrado ou espadelado, mas não fiado",produtos basicos,"Linho em bruto ou trabalhado, mas não fiado; r...",1990,Demais produtos básicos,1990,Demais produtos básicos,Preparação e fiação de fibras têxteis
1,53012110,Linho quebrado,530121,Outras fibras têxteis vegetais; fios de papel ...,"Linho em bruto ou trabalhado, mas não fiado; e...","Linho quebrado ou espadelado, mas não fiado",produtos basicos,"Linho em bruto ou trabalhado, mas não fiado; r...",1990,Demais produtos básicos,1990,Demais produtos básicos,Preparação e fiação de fibras têxteis
2,71023100,"Diamantes não industriais, em bruto ou simples...",710231,"Pérolas naturais ou cultivadas, pedras precios...","Diamantes, mesmo trabalhados, mas não montados...","Diamantes não industriais, em bruto ou serrado...",produtos basicos,Diamantes (exceto diamantes industriais ordena...,1275,"Pedras preciosas ou semipreciosas,em bruto,não...",1990,Demais produtos básicos,Fabricação de joalharia e artigos afins
3,79020000,"Desperdícios e resíduos, de zinco",790200,Zinco e suas obras,"Desperdícios e resíduos, de zinco","Desperdícios e resíduos, de zinco",produtos basicos,Resíduos e sucata de zinco (exceto pó),1990,Demais produtos básicos,1990,Demais produtos básicos,Não classificado
4,81083000,Desperdícios e resíduos do titânio,810830,Outros metais comuns; ceramais (cermets); obra...,"Titânio e suas obras, incluídos os desperdício...",Desperdícios e resíduos do titânio,produtos basicos,Titânio em formas brutas; resíduos e sucata de...,1990,Demais produtos básicos,1990,Demais produtos básicos,Fabricação de metais preciosos e outros metais...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13148,99980101,Consumo de bordo - combustíveis e lubrificante...,999801,Transações especiais,Consumo de bordo (combustíveis e lubrificantes...,Consumo de bordo (combustíveis e lubrificantes...,consumo de bordo,Operações especiais e commodities não classifi...,5020,Consumo de bordo - óleos e combustíveis,4999,NCM sem PPI atribuído,Não classificado
13149,99980202,Consumo de bordo - qualquer outra mercadoria p...,999802,Transações especiais,Consumo de bordo (combustíveis e lubrificantes...,Consumo de bordo para embarcações e aeronaves ...,consumo de bordo,Operações especiais e commodities não classifi...,5010,Consumo de bordo - demais,4999,NCM sem PPI atribuído,Não classificado
13150,99980201,Consumo de bordo - qualquer outra mercadoria p...,999802,Transações especiais,Consumo de bordo (combustíveis e lubrificantes...,Consumo de bordo para embarcações e aeronaves ...,consumo de bordo,Operações especiais e commodities não classifi...,5010,Consumo de bordo - demais,4999,NCM sem PPI atribuído,Não classificado
13151,99999951,Devolução antes da DI (exportação definitiva),999999,Transações especiais,Consumo de bordo (exceto combustíveis e lubrif...,Qualquer outra mercadoria sem cobertura cambia...,reexportacao,Operações especiais e commodities não classifi...,6010,Reexportação,4999,NCM sem PPI atribuído,Não classificado


### **Gravação dos arquivos na camada silver**
---
**Arquivos gravados e convertidos para forma parquet, compressão e redução do tamanho do arquivo**

In [ ]:
NCM_FULL_JOIN.write.parquet("s3a://silver/NCM_FULL_JOIN.parquet", mode="overwrite")

NCM.write.parquet("s3a://silver/NCM.parquet", mode="overwrite")
NCM_SH.write.parquet("s3a://silver/NCM_SH.parquet", mode="overwrite")
NCM_CUCI.write.parquet("s3a://silver/NCM_CUCI.parquet", mode="overwrite")
NCM_CGCE.write.parquet("s3a://silver/NCM_CGCE.parquet", mode="overwrite")
NCM_ISIC.write.parquet("s3a://silver/NCM_ISIC.parquet" , mode="overwrite")
NCM_FAT_AGREG.write.parquet("s3a://silver/NCM_FAT_AGREG.parquet", mode="overwrite")
NBM_NCM.write.parquet("s3a://silver/NBM_NCM.parquet" , mode="overwrite")
NCM_UNIDADE.write.parquet("s3a://silver/NCM_UNIDADE.parquet", mode="overwrite")
NCM_CUCI.write.parquet("s3a://silver/NCM_CUCI.parquet" , mode="overwrite")
NCM_PPE.write.parquet("s3a://silver/NCM_PPE.parquet", mode="overwrite")
NCM_PPI.write.parquet("s3a://silver/NCM_PPI.parquet" , mode="overwrite")
ISIC_CUCI.write.parquet("s3a://silver/ISIC_CUCI.parquet" , mode="overwrite")
UF_MUN.write.parquet("s3a://silver/UF_MUN.parquet", mode="overwrite")
PAIS.write.parquet("s3a://silver/PAIS.parquet" , mode="overwrite")
VIA.write.parquet("s3a://silver/VIA.parquet", mode="overwrite")
PAIS_BLOCO.write.parquet("s3a://silver/PAIS_BLOCO.parquet", mode="overwrite")


### **Finalização das instancias do Spark e liberação da memória**

In [31]:
spark_settings.stop()

##  **2.0 Carregamento dos datasets completos da balança comercial**


###  **2.1 Inicialização de novas instancias do Spark**

In [3]:
spark_settings = SettingsSpark(instance="4", executor_memory="512m")
spark_settings.init_minio()
spark = spark_settings.builder_conf()

###  **2.2 Carregamento**

In [4]:
EXP_COMPLETA = spark.read.csv("s3a://bronze/EXP_COMPLETA.csv", sep=';', inferSchema=True, header=True)
IMP_COMPLETA = spark.read.csv("s3a://bronze/IMP_COMPLETA.csv", sep=';', inferSchema=True, header=True)

EXP_COMPLETA.printSchema()
IMP_COMPLETA.printSchema()

root
 |-- CO_ANO: integer (nullable = true)
 |-- CO_MES: integer (nullable = true)
 |-- CO_NCM: integer (nullable = true)
 |-- CO_UNID: integer (nullable = true)
 |-- CO_PAIS: integer (nullable = true)
 |-- SG_UF_NCM: string (nullable = true)
 |-- CO_VIA: integer (nullable = true)
 |-- CO_URF: integer (nullable = true)
 |-- QT_ESTAT: long (nullable = true)
 |-- KG_LIQUIDO: long (nullable = true)
 |-- VL_FOB: integer (nullable = true)

root
 |-- CO_ANO: integer (nullable = true)
 |-- CO_MES: integer (nullable = true)
 |-- CO_NCM: integer (nullable = true)
 |-- CO_UNID: integer (nullable = true)
 |-- CO_PAIS: integer (nullable = true)
 |-- SG_UF_NCM: string (nullable = true)
 |-- CO_VIA: integer (nullable = true)
 |-- CO_URF: integer (nullable = true)
 |-- QT_ESTAT: integer (nullable = true)
 |-- KG_LIQUIDO: integer (nullable = true)
 |-- VL_FOB: integer (nullable = true)
 |-- VL_FRETE: integer (nullable = true)
 |-- VL_SEGURO: integer (nullable = true)



### **Harmonização das colunas**

In [5]:
EXP_COMPLETA = EXP_COMPLETA.withColumn("VL_FRETE",lit("0")) \
                           .withColumn("VL_SEGURO",lit("0")) \
                           .withColumn("TYPE",lit("EXP"))
        
IMP_COMPLETA = IMP_COMPLETA.withColumn("TYPE",lit("IMP"))

#EXP_COMPLETA.printSchema()
EXP_COMPLETA.show(10)
#IMP_COMPLETA.printSchema()
IMP_COMPLETA.show(10)

IMP_COMPLETA.printSchema()


+------+------+--------+-------+-------+---------+------+-------+--------+----------+------+--------+---------+----+
|CO_ANO|CO_MES|  CO_NCM|CO_UNID|CO_PAIS|SG_UF_NCM|CO_VIA| CO_URF|QT_ESTAT|KG_LIQUIDO|VL_FOB|VL_FRETE|VL_SEGURO|TYPE|
+------+------+--------+-------+-------+---------+------+-------+--------+----------+------+--------+---------+----+
|  1997|    11|74122000|     10|    249|       SP|     4| 817600|       0|         0|     3|       0|        0| EXP|
|  1997|    11|73082000|     10|     97|       MG|     7| 130151|       0|      4292|  6442|       0|        0| EXP|
|  1997|    11|84248119|     11|    196|       SP|     1| 817800|      32|      3468| 31510|       0|        0| EXP|
|  1997|    11|48239090|     10|    586|       MN|     9| 910600|       0|         0|     1|       0|        0| EXP|
|  1997|    11|72112910|     10|    850|       SP|     1| 817800|     118|    117505|103866|       0|        0| EXP|
|  1997|    11|84483919|     10|     63|       SP|     4| 817600

+------+------+--------+-------+-------+---------+------+-------+--------+----------+------+--------+---------+----+
|CO_ANO|CO_MES|  CO_NCM|CO_UNID|CO_PAIS|SG_UF_NCM|CO_VIA| CO_URF|QT_ESTAT|KG_LIQUIDO|VL_FOB|VL_FRETE|VL_SEGURO|TYPE|
+------+------+--------+-------+-------+---------+------+-------+--------+----------+------+--------+---------+----+
|  1997|     9|29251990|     10|     87|       SP|     1| 817800|     200|       200|  4888|     180|       24| IMP|
|  1997|     8|85049010|     10|    399|       SP|     4| 817600|    4891|        84| 30116|     409|       93| IMP|
|  1997|     8|54023300|     10|     63|       SP|     7|1010900|  143834|    143834|402427|   10121|      266| IMP|
|  1997|     3|38121000|     10|    249|       SP|     1| 817800|    9434|      9172| 61190|    3011|      163| IMP|
|  1997|    10|73209000|     10|    399|       SP|     4| 817600|  222510|        49|  3638|     456|       18| IMP|
|  1997|     4|40132000|     11|    161|       SC|     4| 817600

### **Reescrita dos dados harmonizados e ajustados no fomato csv**

In [ ]:
EXP_COMPLETA.write.csv("s3a://silver/EXP_COMPLETA.csv", mode="overwrite")
IMP_COMPLETA.write.csv("s3a://silver/IMP_COMPLETA.csv", mode="overwrite")

### **Junção dos datasets de exportação e importação**

In [ ]:
df_exp_imp_full = EXP_COMPLETA.union(IMP_COMPLETA).distinct()
df_exp_imp_full.show()

In [ ]:
df_exp_imp_full.count()

### **Escrita dos dados exportação e importação após a junção no formato parquet**

In [ ]:
df_exp_imp_full.write.parquet("s3a://silver/EXP_IMP_FULL.parquet", mode="overwrite")

### **Finalização das instancias do Spark e liberação da memória**

In [18]:
spark_settings.stop()

# **Carregamento dos dados completos no formato parquet**

### **Inicialização de novas instancias do Spark**

In [19]:
spark_settings = SettingsSpark(instance="4", executor_memory="512m")
spark_settings.init_minio()
spark = spark_settings.builder_conf()

In [20]:
EXP_IMP_FULL = spark.read.parquet("s3a://silver/EXP_IMP_FULL.parquet", sep=';', inferSchema=True, header=True)
EXP_IMP_FULL.show()


+------+------+--------+-------+-------+---------+------+------+--------+----------+------+--------+---------+----+
|CO_ANO|CO_MES|  CO_NCM|CO_UNID|CO_PAIS|SG_UF_NCM|CO_VIA|CO_URF|QT_ESTAT|KG_LIQUIDO|VL_FOB|VL_FRETE|VL_SEGURO|TYPE|
+------+------+--------+-------+-------+---------+------+------+--------+----------+------+--------+---------+----+
|  2014|    11| 7123900|     10|    158|       SP|     4|817600|      16|        16|     0|     752|        0| IMP|
|  2021|     3| 7133190|     10|    676|       MT|     4|817600|       0|         0|     0|       0|        0| EXP|
|  2019|     8| 8119000|     10|    383|       PA|     4|817600|       2|         2|     0|       0|        0| EXP|
|  2016|     8| 8134090|     10|     87|       PA|     4|817600|       0|         0|     0|       0|        0| EXP|
|  2019|    11| 9019000|     10|    361|       SP|     4|817700|       0|         0|     0|       0|        0| EXP|
|  2021|     5| 9042200|     10|     23|       SP|     4|817600|       1

In [ ]:
"""from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# Trains a k-means model.
kmeans = KMeans().setK(2).setSeed(1)
model = kmeans.fit(EXP_IMP_FULL)

# Make predictions
predictions = model.transform(EXP_IMP_FULL)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)"""

In [28]:
import matplotlib.pyplot

In [29]:
EXP_IMP_FULL.summary().show()

+-------+------------------+-----------------+-------------------+------------------+------------------+---------+-----------------+-----------------+--------------------+------------------+------------------+-----------------+------------------+--------+
|summary|            CO_ANO|           CO_MES|             CO_NCM|           CO_UNID|           CO_PAIS|SG_UF_NCM|           CO_VIA|           CO_URF|            QT_ESTAT|        KG_LIQUIDO|            VL_FOB|         VL_FRETE|         VL_SEGURO|    TYPE|
+-------+------------------+-----------------+-------------------+------------------+------------------+---------+-----------------+-----------------+--------------------+------------------+------------------+-----------------+------------------+--------+
|  count|          59727398|         59727398|           59727398|          59727398|          59727398| 59727398|         59727398|         59727398|            59727398|          59727398|          59727398|         59727398|     

In [30]:
NCM_FULL_JOIN = spark.read.parquet("s3a://silver/NCM_FULL_JOIN.parquet")

In [33]:
NCM_FULL_JOIN.show()

+--------+--------------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+------+--------------------+------+--------------------+--------------------+
|  CO_NCM|              NO_NCM|CO_SH6|          NO_SH2_POR|          NO_SH4_POR|              NO_SH6|        NO_FAT_AGREG|        NO_CUCI_ITEM|CO_PPE|          NO_PPE_MIN|CO_PPI|          NO_PPI_MIN|      NO_ISIC_CLASSE|
+--------+--------------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+------+--------------------+------+--------------------+--------------------+
|38085910|Outras mercadoria...|380859|Produtos diversos...|Insecticidas, rod...|Outras mercadoria...|produtos manufatu...|Outros produtos d...|  3293|Inseticidas, form...|  3293|Inseticidas, form...|Fabricação de pes...|
|38085921|Mercadorias à bas...|380859|Produtos diversos...|Insecticidas, rod...|Outras mercadoria...|produtos manufa

In [41]:
NCM_FULL_JOIN.groupBy("NO_SH2_POR").count().sort(col("count").desc()).show()

+--------------------+-----+
|          NO_SH2_POR|count|
+--------------------+-----+
|Produtos químicos...| 1947|
|Reatores nucleare...| 1335|
|Máquinas, aparelh...|  991|
|Peixes e crustáce...|  500|
|Produtos químicos...|  495|
|Instrumentos e ap...|  461|
|Produtos farmacêu...|  456|
|Produtos diversos...|  408|
|Plásticos e suas ...|  361|
|Papel e cartão; o...|  257|
|                  29|  251|
|Ferro fundido, fe...|  219|
|Madeira, carvão v...|  188|
|Veículos automóve...|  181|
|Obras de ferro fu...|  180|
|Fibras sintéticas...|  179|
|Vestuário e seus ...|  161|
|             Algodão|  160|
|Borracha e suas o...|  138|
|Peles, exceto as ...|  138|
+--------------------+-----+
only showing top 20 rows



In [44]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Load training data
data = NCM_FULL_JOIN

# Split the data into train and test
splits = data.randomSplit([0.6, 0.4], 1234)


train = splits[0]
test = splits[1]

# create the trainer and set its parameters
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")

# train the model
model = nb.fit(train)

# select example rows to display.
predictions = model.transform(test)
predictions.show()

# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

AttributeError: 'NaiveBayes' object has no attribute 'fitMultiplet'